In [37]:
import numpy as np
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv1D, \
    AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout, Softmax
from tensorflow.keras.models import Sequential
from keras.models import Model
from keras.initializers import glorot_uniform
from tensorflow import keras
from numpy import genfromtxt
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.keras.callbacks import TensorBoard
import os

from keras import backend as K
from tensorflow.keras.optimizers import Adam

In [38]:
# SNR
Training_Data = genfromtxt('Training_Data.csv', delimiter=",")
Training_Data = np.array(Training_Data)
#Training_Data = Training_Data[...,None]
print(Training_Data.shape)

Training_Labels = genfromtxt('Training_Labels.csv', delimiter=",")
Training_Labels = np.array(Training_Labels)
#Training_Labels = Training_Labels[...,None]
print(Training_Labels.shape)

Testing_Data = genfromtxt('Testing_Data.csv', delimiter=",")
Testing_Data = np.array(Testing_Data)
#Testing_Data = Testing_Data[...,None]
print(Testing_Data.shape)

Testing_Labels = genfromtxt('Testing_Labels.csv', delimiter=",")
Testing_Labels = np.array(Testing_Labels)
#Testing_Labels = Testing_Labels[...,None]
print(Testing_Labels.shape)


indices_1 = np.arange(Training_Data.shape[0])
np.random.shuffle(indices_1)
Training_Data = Training_Data[indices_1]
Training_Labels = Training_Labels[indices_1]

indices_2 = np.arange(Testing_Data.shape[0])
np.random.shuffle(indices_2)
Testing_Data = Testing_Data[indices_2]
Testing_Labels = Testing_Labels[indices_2]

(700000, 60)
(700000, 90)
(18400, 60)
(18400, 90)


In [39]:
metrics = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'),
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    keras.metrics.F1Score(name='f1_score'),

]

In [40]:
# Define the model architecture
model = Sequential([
    Dense(240, activation='relu', input_shape=(60,)),
    BatchNormalization(),
    Dropout(0.2),
    Dense(120, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(60, activation='relu'),
    BatchNormalization(),
    Dense(90, activation='sigmoid')  # Sigmoid activation for multi-label output
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',metrics=metrics)


In [41]:
batch = 500
epoch = 20

history = model.fit(Training_Data, Training_Labels, validation_split=0.01, epochs=epoch, batch_size=batch,
                      shuffle=True, verbose=1)
preds = model.evaluate(Testing_Data, Testing_Labels)


Epoch 1/20
1386/1386 [==============================] - 8s 4ms/step - loss: 0.1441 - tp: 3868972.0000 - fp: 1795955.0000 - tn: 54337064.0000 - fn: 2368028.0000 - accuracy: 0.9332 - precision: 0.6830 - recall: 0.6203 - auc: 0.9642 - f1_score: 0.0388 - val_loss: 0.0209 - val_tp: 57187.0000 - val_fp: 194.0000 - val_tn: 566806.0000 - val_fn: 5813.0000 - val_accuracy: 0.9905 - val_precision: 0.9966 - val_recall: 0.9077 - val_auc: 0.9996 - val_f1_score: 0.0419
Epoch 2/20
1386/1386 [==============================] - 6s 4ms/step - loss: 0.0190 - tp: 5935582.0000 - fp: 155281.0000 - tn: 55977752.0000 - fn: 301418.0000 - accuracy: 0.9927 - precision: 0.9745 - recall: 0.9517 - auc: 0.9988 - f1_score: 0.0425 - val_loss: 0.0040 - val_tp: 62470.0000 - val_fp: 58.0000 - val_tn: 566942.0000 - val_fn: 530.0000 - val_accuracy: 0.9991 - val_precision: 0.9991 - val_recall: 0.9916 - val_auc: 0.9999 - val_f1_score: 0.0395
Epoch 3/20
1386/1386 [==============================] - 6s 4ms/step - loss: 0.0083 - t

In [42]:
print("loss = " + str(preds[0]))
print("TP = " + str(preds[1]))
print("FP = " + str(preds[2]))
print("TN = " + str(preds[3]))
print("FN = " + str(preds[4]))
print("BinaryAccuracy = " + str(preds[5]))
print("Precision = " + str(preds[6]))
print("Recall = " + str(preds[7]))
print("AUC = " + str(preds[8]))
#print("f1_score = " + str(preds[9]))


model.summary()

loss = 0.0015285570407286286
TP = 164833.0
FP = 2.0
TN = 1490398.0
FN = 767.0
BinaryAccuracy = 0.9995356202125549
Precision = 0.9999878406524658
Recall = 0.9953683614730835
AUC = 0.999648928642273
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 240)               14640     
                                                                 
 batch_normalization_18 (Ba  (None, 240)               960       
 tchNormalization)                                               
                                                                 
 dropout_12 (Dropout)        (None, 240)               0         
                                                                 
 dense_25 (Dense)            (None, 120)               28920     
                                                                 
 batch_normalization_19 (Ba  (None, 120)               